In [2]:
import geopandas as gpd
import pandas as pd
import sqlalchemy as sal
from sqlalchemy import create_engine

In [19]:
engine = sal.create_engine("postgresql://alvessr@localhost/svosm")
conn = engine.connect()

In [8]:
!pwd

/home/alvessr/Repos/OSM-SouthBay/SJ_Buildings


In [9]:
buildings = gpd.read_file('data/BuildingFootprint.geojson')

In [10]:
addresses = gpd.read_file('data/Site_Address_Points.geojson')

In [11]:
parcels = gpd.read_file('data/Parcel.geojson')

In [12]:
condoparcels = gpd.read_file('data/CondoParcel.geojson')

In [20]:
# Connect to database using a context manager
gdf.to_sql(table_name, con=conn, if_exists='append', index=False)

In [13]:
def wkb_hexer(line):
    return line.wkb_hex

gdf = buildings
# Convert `'geom'` column in GeoDataFrame `gdf` to hex
# Note that following this step, the GeoDataFrame is just a regular DataFrame
# because it does not have a geometry column anymore. Also note that
# it is assumed the `'geom'` column is correctly datatyped.
gdf['geom'] = gdf['geometry'].apply(wkb_hexer)

# Delete extra WKT geometry colum.
del gdf['geometry']

# Empty column.
del gdf['LENGTH']
del gdf['SHAPE_AREA']

table_name = 'buildings'

# Connect to database using a context manager
gdf.to_sql(table_name, con=conn, if_exists='append', index=False)

In [21]:
#C onvert the geom column to a geometry.
sql = """ALTER TABLE %s
ALTER COLUMN geom TYPE Geometry(MULTIPOLYGON, 4326)
                  USING ST_SetSRID(geom::Geometry, 4326)""" % (table_name)
conn.execute(sql)